# Probabilistic metric integration
After developing probabilistic metrics in #2232 need to ensure they are compatible with useful features such as grid search for model parameters. There are two key problems that need to be solved for this:

1. proba metrics take in the output of `predict_quantile` or `predict_interval` (or `predict_proba`) where normal metrics just take predict. This means we need to change what predictions are used inside the grid search.

2. Some probabilistic metrics have their own hyperparameters. For example the quantile used in a pinball loss. Currently this is inferred from the data inputted, however for a grid search we will need to somehow tell it what quantile to produce. 

To solve 1. could either create some `set_default` function which determines what the forecaster implements for predict (_predict, _predict_quantile or _predict_interval) or use tags inside the grid search evaluation that retrieves the type of metric being used and calls the corresponding predict function.

To solve 2. we could do a small refactor to the probabilistic metrics, where we specify the hyperprameter(s) we want and it retrieves the correct data from the input (and raises an error if it isn't there). This will allow it to require a specific quantile but reduces flexibility as a user will have to instantiate a new metric class for each different set of quantiles they want to evaluate.

In [1]:
# Basic imports
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import numpy as np
import pandas as pd

In [2]:
# Prep data/forecaster
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.theta import ThetaForecaster

y = np.log1p(load_airline())
y_train, y_test = temporal_train_test_split(y)
fh = np.arange(len(y_test)) + 1

f = ThetaForecaster(sp=12)
f.fit(y_train)
y_pred = f.predict(fh=fh)
q_pred = f.predict_quantiles(fh=fh, alpha=[0.05, 0.1])
i_pred = f.predict_interval(fh=fh)
q_pred

Quantiles          
             0.05      0.10
1958-01  5.771386  5.788262
1958-02  5.750227  5.770302
1958-03  5.894854  5.917684
1958-04  5.839605  5.864893
1958-05  5.826123  5.853649
1958-06  5.953705  5.983301
1958-07  6.049360  6.080890
1958-08  6.033263  6.066615
1958-09  5.897232  5.932312
1958-10  5.737973  5.774699
1958-11  5.578756  5.617057
1958-12  5.717577  5.757392
1959-01  5.726268  5.767541
1959-02  5.713084  5.755765
1959-03  5.865705  5.909750
1959-04  5.815041  5.860407
1959-05  5.805777  5.852429
1959-06  5.938531  5.986433
1959-07  6.038524  6.087644
1959-08  6.025142  6.075452
1959-09  5.890189  5.941660
1959-10  5.731497  5.784103
1959-11  5.572635  5.626354
1959-12  5.714929  5.769737
1960-01  5.725494  5.781370
1960-02  5.713805  5.770729
1960-03  5.869630  5.927584
1960-04  5.819820  5.878785
1960-05  5.811774  5.871733
1960-06  5.947223  6.008160
1960-07  6.049473  6.111372
1960-08  6.037033  6.099880
1960-09  5.901626  5.965406
1960-10  5.742167  5.806867
1960-11  5.582488  5.648095
1960-12  5.727215  5.793717

In [22]:
from sktime.performance_metrics.forecasting import mean_absolute_error

# Define probabilistic metric
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss

loss = PinballLoss(score_average=True, alpha=[0.05, 0.1])
loss2 = mean_absolute_error()

In [4]:
loss(y_true=y_test, y_pred=q_pred)

0    0.014608
dtype: float64

In [5]:
forecaster = f

In [6]:
pred_type = {
    "pred_quantiles": "forecaster.predict_quantiles",
    "pred_intervals": "forecaster.predict_interval",
    "pred_proba": "forecaster.predict_proba",
    None: "forecaster.predict",
}

In [7]:
eval(pred_type["pred_quantiles"])(fh=fh)

Quantiles          
             0.05      0.95
1958-01  5.771386  5.924195
1958-02  5.750227  5.932007
1958-03  5.894854  6.101584
1958-04  5.839605  6.068584
1958-05  5.826123  6.075371
1958-06  5.953705  6.221694
1958-07  6.049360  6.334863
1958-08  6.033263  6.335266
1958-09  5.897232  6.214878
1958-10  5.737973  6.070528
1958-11  5.578756  5.925579
1958-12  5.717577  6.078104
1959-01  5.726268  6.099997
1959-02  5.713084  6.099564
1959-03  5.865705  6.264529
1959-04  5.815041  6.225837
1959-05  5.805777  6.228208
1959-06  5.938531  6.372283
1959-07  6.038524  6.483310
1959-08  6.025142  6.480695
1959-09  5.890189  6.356259
1959-10  5.731497  6.207853
1959-11  5.572635  6.059060
1959-12  5.714929  6.211218
1960-01  5.725494  6.231455
1960-02  5.713805  6.229256
1960-03  5.869630  6.394400
1960-04  5.819820  6.353746
1960-05  5.811774  6.354702
1960-06  5.947223  6.499006
1960-07  6.049473  6.609971
1960-08  6.037033  6.606113
1960-09  5.901626  6.479159
1960-10  5.742167  6.328032
1960-11  5.582488  6.176568
1960-12  5.727215  6.329399

In [8]:
f.predict_quantiles()

Quantiles          
             0.05      0.95
1958-01  5.771386  5.924195
1958-02  5.750227  5.932007
1958-03  5.894854  6.101584
1958-04  5.839605  6.068584
1958-05  5.826123  6.075371
1958-06  5.953705  6.221694
1958-07  6.049360  6.334863
1958-08  6.033263  6.335266
1958-09  5.897232  6.214878
1958-10  5.737973  6.070528
1958-11  5.578756  5.925579
1958-12  5.717577  6.078104
1959-01  5.726268  6.099997
1959-02  5.713084  6.099564
1959-03  5.865705  6.264529
1959-04  5.815041  6.225837
1959-05  5.805777  6.228208
1959-06  5.938531  6.372283
1959-07  6.038524  6.483310
1959-08  6.025142  6.480695
1959-09  5.890189  6.356259
1959-10  5.731497  6.207853
1959-11  5.572635  6.059060
1959-12  5.714929  6.211218
1960-01  5.725494  6.231455
1960-02  5.713805  6.229256
1960-03  5.869630  6.394400
1960-04  5.819820  6.353746
1960-05  5.811774  6.354702
1960-06  5.947223  6.499006
1960-07  6.049473  6.609971
1960-08  6.037033  6.606113
1960-09  5.901626  6.479159
1960-10  5.742167  6.328032
1960-11  5.582488  6.176568
1960-12  5.727215  6.329399

In [9]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
)

cv = ExpandingWindowSplitter(
    initial_window=24, step_length=12, fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
)

param_grid = {"sp": [6, 12]}

gcv = ForecastingGridSearchCV(f, cv, param_grid, scoring=loss)

In [23]:
loss = PinballLoss(score_average=True, alpha=[0.05, 0.5, 0.95])
gcv = ForecastingGridSearchCV(f, cv, param_grid, scoring=loss)
gcv2 = ForecastingGridSearchCV(f, cv, param_grid, scoring=loss2)

In [24]:
out = evaluate(f, cv, y_test, scoring=loss2)

AttributeError: 'function' object has no attribute 'get_class_tag'

In [19]:
out

,test_PinballLoss,fit_time,pred_time,len_train_window,cutoff
0,0 0.013577 dtype: float64,0.008752,0.007927,24,1959-12


In [20]:
out.mean()

test_PinballLoss     0.013577
fit_time             0.008752
pred_time            0.007927
len_train_window    24.000000
dtype: float64

In [14]:
print(loss.alpha)

[0.05, 0.5, 0.95]


In [15]:
print(loss.metric_args)

{'alpha': [0.05, 0.5, 0.95]}


In [21]:
gcv.fit(y_test)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            initial_window=None, step_length=12),
                        forecaster=ThetaForecaster(sp=12),
                        param_grid={'sp': [6, 12]},
                        scoring=PinballLoss(alpha=[0.05, 0.5, 0.95],
                                            score_average=True))

The ForecastingGridSearchCV relies on `sktime.forecasting.model_evaluation.evaluate` to evaluate metric scores, hence this is what we will need to change to allow it to work. It also has it's own `score()` function which could also be changed but this isn't used in fitting.

In [ ]:
f.predict_quantiles()

Quantiles          
             0.05      0.95
1958-01  5.771386  5.924195
1958-02  5.750227  5.932007
1958-03  5.894854  6.101584
1958-04  5.839605  6.068584
1958-05  5.826123  6.075371
1958-06  5.953705  6.221694
1958-07  6.049360  6.334863
1958-08  6.033263  6.335266
1958-09  5.897232  6.214878
1958-10  5.737973  6.070528
1958-11  5.578756  5.925579
1958-12  5.717577  6.078104
1959-01  5.726268  6.099997
1959-02  5.713084  6.099564
1959-03  5.865705  6.264529
1959-04  5.815041  6.225837
1959-05  5.805777  6.228208
1959-06  5.938531  6.372283
1959-07  6.038524  6.483310
1959-08  6.025142  6.480695
1959-09  5.890189  6.356259
1959-10  5.731497  6.207853
1959-11  5.572635  6.059060
1959-12  5.714929  6.211218
1960-01  5.725494  6.231455
1960-02  5.713805  6.229256
1960-03  5.869630  6.394400
1960-04  5.819820  6.353746
1960-05  5.811774  6.354702
1960-06  5.947223  6.499006
1960-07  6.049473  6.609971
1960-08  6.037033  6.606113
1960-09  5.901626  6.479159
1960-10  5.742167  6.328032
1960-11  5.582488  6.176568
1960-12  5.727215  6.329399

In [ ]:
evaluate(f, cv, y_test, scoring=loss)

ValueError: not all quantile values in alpha are available in y_pred

If we naively substitute the normal loss for a quantile loss we get an input error (as expected).

We will first try changing the evaluate function.

In [ ]:
import time

from sklearn.base import clone

from sktime.forecasting.base import ForecastingHorizon
from sktime.utils.validation.forecasting import (
    check_cv,
    check_fh,
    check_scoring,
    check_X,
)
from sktime.utils.validation.series import check_series


def evaluate(
    forecaster,
    cv,
    y,
    X=None,
    strategy="refit",
    scoring=None,
    fit_params=None,
    return_data=False,
):
    """Evaluate forecaster using timeseries cross-validation.

    Parameters
    ----------
    forecaster : sktime.forecaster
        Any forecaster
    cv : Temporal cross-validation splitter
        Splitter of how to split the data into test data and train data
    y : pd.Series
        Target time series to which to fit the forecaster.
    X : pd.DataFrame, default=None
        Exogenous variables
    strategy : {"refit", "update"}
        Must be "refit" or "update". The strategy defines whether the `forecaster` is
        only fitted on the first train window data and then updated, or always refitted.
    scoring : subclass of sktime.performance_metrics.BaseMetric, default=None.
        Used to get a score function that takes y_pred and y_test arguments
        and accept y_train as keyword argument.
        If None, then uses scoring = MeanAbsolutePercentageError(symmetric=True).
    fit_params : dict, default=None
        Parameters passed to the `fit` call of the forecaster.
    return_data : bool, default=False
        Returns three additional columns in the DataFrame, by default False.
        The cells of the columns contain each a pd.Series for y_train,
        y_pred, y_test.

    Returns
    -------
    pd.DataFrame
        DataFrame that contains several columns with information regarding each
        refit/update and prediction of the forecaster.
    """
    _check_strategy(strategy)
    cv = check_cv(cv, enforce_start_with_window=True)
    scoring = check_scoring(scoring)
    y = check_series(
        y,
        enforce_univariate=forecaster.get_tag("scitype:y") == "univariate",
        enforce_multivariate=forecaster.get_tag("scitype:y") == "multivariate",
    )
    X = check_X(X)
    fit_params = {} if fit_params is None else fit_params

    # Define score name.
    score_name = "test_" + scoring.name

    # Initialize dataframe.
    results = pd.DataFrame()

    # Run temporal cross-validation.
    for i, (train, test) in enumerate(cv.split(y)):
        # split data
        y_train, y_test, X_train, X_test = _split(y, X, train, test, cv.fh)

        # create forecasting horizon
        fh = ForecastingHorizon(y_test.index, is_relative=False)

        # fit/update
        start_fit = time.perf_counter()
        if i == 0 or strategy == "refit":
            forecaster = clone(forecaster)
            forecaster.fit(y_train, X_train, fh=fh, **fit_params)

        else:  # if strategy == "update":
            forecaster.update(y_train, X_train)
        fit_time = time.perf_counter() - start_fit

        # predict
        start_pred = time.perf_counter()
        if scoring.get_tag("scitype:y_pred") == "pred_quantiles":
            y_pred = forecaster.predict_quantiles(fh, X=X_test, **fit_params)
        else:
            y_pred = forecaster.predict(fh, X=X_test)

        pred_time = time.perf_counter() - start_pred

        # score
        score = scoring(y_test, y_pred, y_train=y_train)

        # save results
        results = results.append(
            {
                score_name: score,
                "fit_time": fit_time,
                "pred_time": pred_time,
                "len_train_window": len(y_train),
                "cutoff": forecaster.cutoff,
                "y_train": y_train if return_data else np.nan,
                "y_test": y_test if return_data else np.nan,
                "y_pred": y_pred if return_data else np.nan,
            },
            ignore_index=True,
        )

    # post-processing of results
    if not return_data:
        results = results.drop(columns=["y_train", "y_test", "y_pred"])
    results["len_train_window"] = results["len_train_window"].astype(int)

    return results


def _split(y, X, train, test, fh):
    """Split y and X for given train and test set indices."""
    y_train = y.iloc[train]
    y_test = y.iloc[test]

    cutoff = y_train.index[-1]
    fh = check_fh(fh)
    fh = fh.to_relative(cutoff)

    if X is not None:
        X_train = X.iloc[train, :]

        # We need to expand test indices to a full range, since some forecasters
        # require the full range of exogenous values.
        test = np.arange(test[0] - fh.min(), test[-1]) + 1
        X_test = X.iloc[test, :]
    else:
        X_train = None
        X_test = None

    return y_train, y_test, X_train, X_test


def _check_strategy(strategy):
    """Assert strategy value.

    Parameters
    ----------
    strategy : str
        strategy of how to evaluate a forecaster

    Raises
    ------
    ValueError
        If strategy value is not in expected values, raise error.
    """
    valid_strategies = ("refit", "update")
    if strategy not in valid_strategies:
        raise ValueError(f"`strategy` must be one of {valid_strategies}")

In [ ]:
evaluate(f, cv, y_test, scoring=loss)

ValueError: not all quantile values in alpha are available in y_pred

## Other solution
A different option would be to change forecasters so that they included a default implementation for predict. 

```{python}
forecaster():
    def __init__(self):
        self.pred_default = "point"
        pred_types = {
            "point":self._predict, 
            "quantile":self._predict_quantile,
            "interval":self._predict_interval}

    def _predict(self, X, fh):
        notimplementederror()

    def _predict_quantile(self, X, fh):
        notimplementederror()

    def _predict_interval(self, X, fh):
        notimplementederror()

    def predict(self, X, fh, type = None):
        if type = None:
            type = self.pred_default

    return pred_types[type](X, fh)
        
```

This would require changing the base class of forecasters and also still wouldn't solve the issue of how we pass what quantiles we need to the predict function.